[Dokumentacja mysql.connector](https://dev.mysql.com/doc/connector-python/en/connector-python-example-connecting.html)

Funkcja connect() tworzy połączenie z MySQL server i zwraca obiekt MySQLConnection:
```python
cnx = mysql.connector.connect(**config_dict)
```

Metoda połączenia cursor() zwraca obiekt MySQLCursor, który umożliwia manipulowanie bazą:
```python
cursor = cnx.cursor()

cursor.execute('sql_statement'[, data])
cursor.executemany('sql_statement', seq_of_data)
# w wyniku zapytania cursor może zostać użyty jako iterator lub można pobrać wyszukane wiersze
row_list = cursor.fetchall()

iterator = cursor.execute('many_sql_statement', multi=True)
```

Po dodaniu (INSERT INTO), zaktualizowaniu (UPDATE) lub usunięciu (DELETE FROM) wierszy, zmiany należy jeszcze skomitować:
```
cnx.commit()
```

Połączenie należy zakończyć:
```
cursor.close()
cnx.close()
```

In [1]:
import mysql.connector
import random

### Nawiązanie połączenia z bazą 

In [2]:
db_name = 'cardb'

In [3]:
# słownik złożony z argumentów kluczowych funkcji connect()
config = {
  'user': 'root',
  'password': 'admin',
  'host': '127.0.0.1',
#   'database': db_name,
}

In [4]:
# połączenie się z serwerem (lub konkretną bazą na tym serwerze, jeżeli przekazany jest też argument database)
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [5]:
# połączenie się z konkretną bazą
# jeżeli baza nie istnieje, zostaje utworzona
try:
    cnx.database = db_name
except mysql.connector.Error as err:
  if err.errno == mysql.connector.errorcode.ER_ACCESS_DENIED_ERROR:
    print('Something is wrong with your user name or password')
  elif err.errno == mysql.connector.errorcode.ER_BAD_DB_ERROR:
#     print('Database {} does not exist'.format(db_name))
    cursor.execute('CREATE DATABASE {}'.format(db_name))
    cnx.database = db_name
  else:
    print(err)

### Utworzenie tabel

In [6]:
table_dict = {}

In [7]:
table_dict['Car'] = '''
    CREATE TABLE `Car` (
    `Id` INT NOT NULL,
    `Model` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`Id`))
    '''

table_dict['Part'] = '''
    CREATE TABLE `Part` (
    `Id` INT NOT NULL,
    `IsFunctional` TINYINT(1) NOT NULL,
    `CarId` INT,
    PRIMARY KEY (`Id`),
    FOREIGN KEY (`CarId`) REFERENCES `Car`(`Id`))
    '''

table_dict['Wheel'] = '''
    CREATE TABLE `Wheel` (
    `PartId` INT NOT NULL,
    `Model` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`PartId`),
    FOREIGN KEY (`PartId`) REFERENCES `Part`(`Id`))
    '''

table_dict['Engine'] = '''
    CREATE TABLE `Engine` (
    `PartId` INT NOT NULL,
    `Model` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`PartId`),
    FOREIGN KEY (`PartId`) REFERENCES `Part`(`Id`))
    '''

In [8]:
# utworzenie tabel
# jeżeli już istnieją, pojawi się komunikat
for table_name, create_table_stmt in table_dict.items():
    try:
        cursor.execute(create_table_stmt)
    except mysql.connector.Error as err:
      if err.errno == mysql.connector.errorcode.ER_TABLE_EXISTS_ERROR:
        print('Table {} already exists'.format(table_name))
      else:
        print(err)

### Dodanie wierszy

In [9]:
add_car_stmt = '''
    INSERT INTO Car (Id, Model)
    VALUES (%s, %s)
    '''

add_part_stmt = '''
    INSERT INTO Part (Id, IsFunctional, CarId)
    VALUES (%s, %s, %s)
    '''

add_wheel_stmt = '''
    INSERT INTO Wheel (PartId, Model)
    VALUES (%s, %s)
    '''

add_engine_stmt = '''
    INSERT INTO Engine (PartId, Model)
    VALUES (%s, %s)
    '''

In [10]:
cursor.execute('SELECT Id FROM Car')
try:
    last_car_id = cursor.fetchall()[-1][0]
except IndexError:
    last_car_id = 0

car_total = 20
car_id_list = list(range(last_car_id + 1, last_car_id + 1 + car_total))
car_model_list = ['Renault', 'Audi', 'BMW', 'Mazda', 'Fiat']
car_list = [(car_id, random.choice(car_model_list)) for car_id in car_id_list]

cursor.execute('SELECT Id FROM Part')
try:
    last_part_id = cursor.fetchall()[-1][0]
except IndexError:
    last_part_id = 0

part_total = 100
part_id_list = list(range(last_part_id + 1, last_part_id + 1 + part_total))
part_list = [(part_id, random.randint(0, 1), random.choice(car_id_list)) for part_id in part_id_list]

wheel_total = int(part_total * 0.7)
wheel_model_list = ['W1', 'W2', 'W3', 'W4', 'W5']
wheel_part_id_list = random.sample(part_id_list, wheel_total)
wheel_list = [(part_id, random.choice(wheel_model_list)) for part_id in wheel_part_id_list]

engine_total = part_total - wheel_total
engine_model_list = ['A157', 'B458', 'A86', 'A123', 'C18']
engine_part_id_list = list(set(part_id_list) - set(wheel_part_id_list))
engine_list = [(part_id, random.choice(engine_model_list)) for part_id in engine_part_id_list]

In [11]:
cursor.executemany(add_car_stmt, car_list)

In [12]:
cursor.executemany(add_part_stmt, part_list)

In [13]:
cursor.executemany(add_wheel_stmt, wheel_list)

In [14]:
cursor.executemany(add_engine_stmt, engine_list)

In [15]:
cnx.commit()

### Zaktualizowanie wierszy

In [16]:
set_free_part_stmt = '''
    UPDATE Part
    SET CarId = NULL
    WHERE Id IN {}
    '''

set_broken_part_stmt = '''
    UPDATE Part
    SET IsFunctional = 0
    WHERE Id = %s
    '''

set_fixed_part_stmt = '''
    UPDATE Part
    SET IsFunctional = 1
    WHERE Id = %s
    '''

In [17]:
free_part_total = 15
free_part_id_list = random.sample(part_id_list, free_part_total)
set_free_part_full_stmt = set_free_part_stmt.format(str(('%s',) * free_part_total))

In [18]:
cursor.execute(set_free_part_full_stmt, free_part_id_list)

In [19]:
fixed_part_id_list = [(part_id,) for part_id in random.sample(part_id_list, 10)]

In [20]:
cursor.executemany(set_fixed_part_stmt, fixed_part_id_list)

In [21]:
cnx.commit()

### Zapytania

In [22]:
def print_res_of_query(cursor):
    frame_str = '{:<10}' * len(cursor.column_names)
    print(frame_str.format(*cursor.column_names))
    for row in cursor:
        print(frame_str.format(*row))    

In [23]:
# znajdź sprawne samochody
# samochód jest sprawny, jeżeli wszystkie jego części są sprawne
get_functional_car_query = '''
    SELECT Car.Id, Model
    FROM Car INNER JOIN Part ON Car.Id = Part.CarId
    GROUP BY Car.Id
    HAVING COUNT(IsFunctional) = SUM(IsFunctional)
    '''

In [24]:
cursor.execute(get_functional_car_query)
print_res_of_query(cursor)

Id        Model     
3         Fiat      
19        Fiat      


In [25]:
# znajdź wszystkie luźne części
# luźne części to te, które nie są związane z żadnym samochodem
get_free_part_query = '''
    SELECT Part.Id AS Id, 'Wheel' AS Type, Model, IsFunctional
    FROM Part INNER JOIN Wheel ON Part.Id = Wheel.PartId
    WHERE CarId IS NULL
    UNION
    SELECT Part.Id AS Id, 'Engine' AS Type, Model, IsFunctional
    FROM Part INNER JOIN Engine ON Part.Id = Engine.PartId
    WHERE CarId IS NULL
    ORDER BY Id
    '''

In [26]:
cursor.execute(get_free_part_query)
print_res_of_query(cursor)

Id        Type      Model     IsFunctional
16        Wheel     W5        0         
17        Wheel     W4        0         
44        Wheel     W5        0         
55        Engine    A123      1         
60        Wheel     W4        1         
63        Wheel     W4        1         
64        Wheel     W5        1         
67        Engine    C18       1         
68        Wheel     W2        0         
75        Wheel     W2        1         
83        Wheel     W3        0         
86        Engine    A123      1         
88        Wheel     W1        1         
90        Wheel     W5        0         
97        Wheel     W1        0         


In [27]:
# znajdź wszystkie części związane z samochodami
get_car_part_query = '''
    SELECT Car.Id AS CarId, Car.Model AS Car, Type AS Part, FullPart.Model AS PartModel, IsFunctional
    FROM Car LEFT JOIN (
        SELECT *, 'Wheel' AS Type
        FROM Part INNER JOIN Wheel ON Part.Id = Wheel.PartId
        UNION
        SELECT *, 'Engine' AS Type
        FROM Part INNER JOIN Engine ON Part.Id = Engine.PartId) AS FullPart 
    ON FullPart.CarId = Car.Id
    LIMIT 20
    '''

In [28]:
cursor.execute(get_car_part_query)
print_res_of_query(cursor)

CarId     Car       Part      PartModel IsFunctional
1         Renault   Engine    B458      1         
1         Renault   Wheel     W2        0         
1         Renault   Wheel     W1        1         
1         Renault   Engine    A123      0         
1         Renault   Engine    A157      1         
1         Renault   Wheel     W1        1         
1         Renault   Engine    A123      0         
2         BMW       Engine    B458      0         
2         BMW       Wheel     W2        1         
2         BMW       Engine    A123      0         
2         BMW       Wheel     W4        1         
2         BMW       Wheel     W2        0         
2         BMW       Wheel     W4        0         
2         BMW       Wheel     W1        0         
2         BMW       Wheel     W2        0         
2         BMW       Wheel     W4        1         
3         Fiat      Wheel     W5        1         
3         Fiat      Engine    A123      1         
3         Fiat      Wheel    

In [29]:
# znajdź dostępne modele i ich liczbę
get_model_query = '''
    SELECT Model, COUNT(*) AS Count FROM {}
    GROUP BY Model
    '''

table_name_list = ['Car', 'Wheel', 'Engine']

In [30]:
result_iter = cursor.execute('; '.join(map(get_model_query.format, table_name_list)), multi=True)
frame_str = '{:%s}: {}' % (max(map(len, table_name_list)) + 1)
for table_name, result in zip(table_name_list, result_iter):
    print(frame_str.format(table_name, result.fetchall()))

Car    : [('Audi', 5), ('BMW', 3), ('Fiat', 4), ('Mazda', 4), ('Renault', 4)]
Wheel  : [('W1', 20), ('W2', 16), ('W3', 5), ('W4', 13), ('W5', 16)]
Engine : [('A123', 11), ('A157', 5), ('A86', 5), ('B458', 4), ('C18', 5)]


### Zakończenie połączenia

In [31]:
cursor.close()
cnx.close()